In [1]:
!pip install -q transformers accelerate sentencepiece

In [2]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM
)
from tqdm import tqdm

In [3]:
CONFIG = {
    "device": "cuda",
    "max_new_tokens": 256,
    "temperature": 0.5,
    "top_p": 0.5,
    "do_sample": True,

    "models": {
        "deepseek": {
            "model_name": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
            "type": "chat"
        },
        "tinyllama": {
            "model_name": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "type": "chat"
        },
        "qwen": {
            "model_name": "Qwen/Qwen2.5-1.5B-Instruct",
            "type": "chat"
        },
        "fred": {
            "model_name": "ai-forever/FRED-T5-1.7B",
            "type": "seq2seq"
        },
        "rut5": {
            "model_name": "ai-forever/ruT5-base",
            "type": "seq2seq"
        },
        "mistralai": {
            "model_name": "mistralai/Mistral-7B-Instruct-v0.3",
            "type": "chat"
        },
        "phi-3-mini": {
            "model_name": "microsoft/Phi-3-mini-4k-instruct",
            "type": "chat"
        }
    }
}


In [14]:
BASE_PROMPT = """
Ты — эксперт по упрощению русских текстов.

Задача:
Адаптируй исходный текст на русском языке до целевого уровня сложности {target_level} по шкале CEFR.

Требования:
- Итоговый текст должен быть на русском языке. Латиница возможна только в именах собственных.
- Текст должен быть максимально близким по содержанию к исходному. Не добавляй и не удаляй важную информацию.
- Соблюдай грамматические, орфографические и пунктуационные нормы русского языка.
- Выведи только итоговый адаптированный текст. Никаких вступлений, комментариев, пояснений или повторений исходного текста.
- Запрещены циклические повторы. Не повторяй одно и то же слово или фразу несколько раз подряд.
- Варьируй структуру предложений: чередуй простые и сложные предложения (в рамках допустимого для целевого уровня), меняй порядок слов для естественности звучания.
- Следи за разнообразием лексики: не используй одно и то же слово в соседних предложениях, если это не ключевой термин.

{additional_instructions}

Исходный текст:
{text}
"""

In [5]:
ADDITIONAL_INSTRUCTIONS = {
    "simplify_more": """
Упрости текст сильнее:
- Замени сложные слова на простые
- Разделяй длинные предложения
- Избегай причастных и деепричастных оборотов
- Используй простые грамматические конструкции
""",

    "simplify_less": """
Не делай текст слишком примитивным:
- Сохрани все важные детали
- Не сокращай предложения слишком сильно
- Не удаляй ключевые элементы содержания
"""
}

In [6]:
LOADED_MODELS = {}

In [7]:
def load_model(model_key):
    """
    Загружает модель один раз и сохраняет в глобальном словаре.
    При повторном вызове возвращает уже загруженную модель.
    """
    if model_key in LOADED_MODELS:
        print(f"Модель {model_key} уже загружена, используется кэш")
        return LOADED_MODELS[model_key]

    print(f"Загрузка модели {model_key}...")
    model_info = CONFIG["models"][model_key]
    model_name = model_info["model_name"]
    model_type = model_info["type"]

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    if model_type == "chat":
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
    else:
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

    model.eval()

    model_data = {
        'tokenizer': tokenizer,
        'model': model,
        'type': model_type,
        'name': model_name
    }

    LOADED_MODELS[model_key] = model_data
    print(f"Модель {model_key} успешно загружена")

    return model_data

In [8]:
def unload_model(model_key):
    """Выгружает модель из памяти"""
    if model_key in LOADED_MODELS:
        if 'model' in LOADED_MODELS[model_key]:
            del LOADED_MODELS[model_key]['model']
        if 'tokenizer' in LOADED_MODELS[model_key]:
            del LOADED_MODELS[model_key]['tokenizer']
        del LOADED_MODELS[model_key]

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print(f"Модель {model_key} выгружена")

In [9]:
def format_prompt(text, target_level="A2", additional_instructions_key=None):
    """
    Формирует полный промпт для генерации текста.
    """
    cefr_levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
    if target_level not in cefr_levels:
        raise ValueError("Переданный уровень не соответствует шкале CEFR")

    additional_instructions = ""
    if additional_instructions_key and additional_instructions_key in ADDITIONAL_INSTRUCTIONS:
        additional_instructions = ADDITIONAL_INSTRUCTIONS[additional_instructions_key]

    prompt = BASE_PROMPT.format(
        target_level=target_level,
        additional_instructions=additional_instructions,
        text=text
    )

    return prompt.strip()

In [10]:
def generate(
    model_key,
    prompt,
    custom_config=None,
    **gen_kwargs
):
    if model_key not in LOADED_MODELS:
        raise ValueError(f"Модель {model_key} не загружена.")

    model_data = LOADED_MODELS[model_key]
    tokenizer = model_data['tokenizer']
    model = model_data['model']
    model_type = model_data['type']

    config_to_use = custom_config if custom_config else CONFIG

    generation_params = {
        'max_new_tokens': config_to_use["max_new_tokens"],
        'temperature': config_to_use["temperature"],
        'top_p': config_to_use["top_p"],
        'do_sample': config_to_use["do_sample"],
    }
    generation_params.update(gen_kwargs)

    with torch.no_grad():
        if model_type == "chat":
            messages = [{"role": "user", "content": prompt}]

            formatted_prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            inputs = tokenizer(
                formatted_prompt,
                return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                **inputs,
                **generation_params,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )

            input_length = inputs.input_ids.shape[1]
            generated_tokens = outputs[0][input_length:]
            generated_text = tokenizer.decode(
                generated_tokens,
                skip_special_tokens=True
            )

        else:
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(model.device)

            outputs = model.generate(
                **inputs,
                **generation_params
            )

            generated_text = tokenizer.decode(
                outputs[0],
                skip_special_tokens=True
            )

    return generated_text.strip()

In [ ]:
# Загрузка моделей
# load_model('deepseek')
# load_model('fred')
# load_model('tinyllama')
# load_model('rut5')

Загрузка модели deepseek...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Модель rut5 успешно загружена


{'tokenizer': T5Tokenizer(name_or_path='ai-forever/ruT5-base', vocab_size=32100, model_max_length=1000000000000000019884624838656, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, added_tokens_decoder={
 	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32000: AddedToken("<extra_id_99>", rstrip=True, lstrip=True, single_word=True, normalized=False, special=True),
 	32001: AddedToken("<extra_id_98>", rstrip=True, lstrip=True, single_word=True, normalized=False, special=True),
 	32002: AddedToken("<extra_id_97>", rstrip=True, lstrip=True, single_word=True, normalized=False, special=True),
 	32003: AddedToken("<extra_id_96>", rstrip=True, lstrip=True

In [ ]:
load_model('qwen')

Загрузка модели qwen...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Модель qwen успешно загружена


{'tokenizer': Qwen2Tokenizer(name_or_path='Qwen/Qwen2.5-1.5B-Instruct', vocab_size=151643, model_max_length=131072, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>'}, added_tokens_decoder={
 	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151647: AddedToken("<|object_ref_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151648: AddedToken("<|box_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151649: AddedToken("<|

In [ ]:
load_model('mistralai')

Загрузка модели mistralai...


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Модель mistralai успешно загружена


{'tokenizer': TokenizersBackend(name_or_path='mistralai/Mistral-7B-Instruct-v0.3', vocab_size=32768, model_max_length=1000000000000000019884624838656, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	6: AddedToken("[AVAILABLE_TOOLS]", rstr

In [11]:
load_model('phi-3-mini')

Загрузка модели phi-3-mini...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Модель phi-3-mini успешно загружена


{'tokenizer': TokenizersBackend(name_or_path='microsoft/Phi-3-mini-4k-instruct', vocab_size=32000, model_max_length=4096, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
 	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
 	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
 	32003: AddedToken("<|pla

In [15]:
prompt = format_prompt(
    text="Квантовая запутанность — физическое явление, при котором квантовые состояния объектов оказываются взаимозависимыми.",
    target_level="A1"
    )

In [ ]:
result = generate('qwen', prompt)
print(result)

Квантовая запутанность — физическое явление, при котором квантовые состояния объектов связаны между собой.


In [ ]:
result1 = generate('fred', prompt)
print(result1)

Объявление в прессе сообщило что компания Pricky Creative, производитель электронных устройств с высокой пропускным способность Интернета вещей EcoLoc , объявила новую премию имени Акаги, победителя предыдущих двух престижианнейших научных конкурсных соревнований и двух престижный профессиональных конкурс на звание "Прометиевым человеком 21/22-века 2020г...


In [ ]:
result2 = generate('rut5', prompt)
print(result2)

: Квантовая запутанность — это такое.)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))


In [ ]:
result3 = generate('tinyllama', prompt)
print(result3)

Разработайте программу для упрощения русских текстов, которая будет работать на языке, использующем латиницу.

1. Загрузите необходимые модули для работы с языком и языковым словарем.

2. Скачайте и установите программу для упрощения русских текстов.

3. Наберите в программу текст, который вы хотите упростить.

4. Выберите язык, который используется в тексте.

5. Нажмите кнопку "Упростить" и просмотрите результат.

6. Если вы хотите, чтобы текст был на русском языке, выберите "Русский" в качестве языка.

7. Перепишите текст на русском языке.

8. Если вы хотите, чтобы текст был на латинице, выберите "Латиница" в качестве языка.

9. Перепишите текст на латинице.

10.


In [ ]:
result4 = generate('deepseek', prompt)
print(result4)

Хм, пользователь попросил переписать текст на русском языке. Я вижу, что исходный текст написан на английском, и он упоминает "квантовая запутанность" как физическое явление, где квантовые состояния объектов оказываются взаимозависимыми. 

Первым делом, я должен понять, что такое квантовая запутанность. Это, наверное, связана с квантовыми физическими процессами, которые не follow classical physics. Возможно, это связано с квантовыми效应ами, которые возникают при небольших масштабах или при высоких energy.

Теперь, чтобы переписать это на русском, я должен сохранить основные идеи: физическое явление, квантовые состояния, взаимозависимость. Возможно, стоит использовать термины, которые уже есть на русском, чтобы упростить текст.

Но я не уверен, как именно отразить это на русском языке.


In [ ]:
result5 = generate('mistralai', prompt)
print(result5)

Квантовая запутанность – это физическое явление, при котором квантовые состояния объектов связаны.


In [16]:
result6 = generate('phi-3-mini', prompt)
print(result6)

Квантовые состояния объектов взаимозависимы.


In [17]:
complicated_text = """
Бо́льшую часть своего правления Филипп II находился в состоянии войны с Афинами, где главой антимакедонской партии был знаменитый оратор Демосфен. Его выступления против македонского царя получили название «филиппик». Впоследствии термин стал синонимом гневных и обличительных речей. После победы над объединённым афино-фиванским войском при Херонее в 338 году до н. э. Филипп II подчинил своему влиянию бо́льшую часть Греции, став гегемоном общеэллинского Коринфского союза. На этом посту он начал готовиться к войне против персов, представив её продолжением греко-персидских войн, которые завершились более чем за сто лет до описываемых событий.
"""

In [18]:
prompt_with_complicated_text = format_prompt(
    text=complicated_text,
    target_level="A1",
    # additional_instructions_key = "simplify_more"
    )

In [ ]:
mistralai_result = generate('mistralai', prompt_with_complicated_text)
print(mistralai_result)

Филипп II большую часть правления находился в войне с Афинами. Главой антимакедонской партии был Демосфен, известный оратор. Его речи против македонского царя называются «филиппиками». Позже это слово стало синонимом гневных и обличительных речей. После победы над объединённым афинско-фиванским войском при Херонее в 338 году до н. э., Филипп II подчинил большую часть Греции своему влиянию и стал гегемоном Коринфского союза. На этом посту он готовился к войне против персов, представив её продолжением греко-персидских войн, которые завершились более чем за сто лет до описываемых событий.


In [ ]:
qwen_result = generate('qwen', prompt_with_complicated_text)
print(qwen_result)

Филипп II во время своей правления постоянно боролся с Афинами. Главным оппонентом Македонии была известный оратор Демосфен. Его выступления против Филиппа называли "филиппик". Этот термин теперь используется для обозначения гневных или обличительных речей. После победы над объединенным афинским и фиванским войсками при Херонее в 338 году до н. э., Филипп II полностью контролировал большую часть Греции, становясь гегемоном общего Эллинского союза. На его посту он начал подготовку к войне с Персиями, считая ее продолжением греко-персидских войн, которые закончились почти сто лет ранее.


In [19]:
phi_3_mini_result = generate('phi-3-mini', prompt_with_complicated_text)
print(phi_3_mini_result)

Филипп II, царь Рима, много лет во время войны с Афинами. В Афины оратор Димоффен opposed Македонский царь. Его речи назывались «филиппиком». После победы на войне в 338 году, царь стал гегемоном Коринфского союза. Он начал подготовиться к войне против Персов, продолжающим греко-персидские войны, которые закончились более чем за сто лет.


In [ ]:
unload_model('qwen')

Модель qwen выгружена


In [ ]:
unload_model('phi-3-mini')

Модель phi-3-mini выгружена


In [ ]:
unload_model('mistralai')

Модель mistralai выгружена
